# Lab 3 Part 2 - Task 1: Parameters in CNN (5 Marks)

- For the model we have created in **Lab 3 Part 1 Exercise**: Early Stopping with Callbacks, calculate the number of parameters by hand for each layer and compare to the output of model.summary() and print the model summary.
- Then print the model summary of **Exercise 7 in Lab 1**
- Now compare the Model you created in **Exercise 7 in Lab 1**,
  - Compare the Parameters of the models

  - Compare Model Performance







In [10]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import GlorotUniform
from tensorflow.keras.datasets import mnist
import numpy as np

(train_data, train_labels), (test_data, test_labels) = mnist.load_data()

In [23]:
# Set the random seed to get same results everytime.
tf.random.set_seed(101)
np.random.seed(101)

In [2]:
# Use the first 10,000 samples of our training data as our validation set
val_data = train_data[:10000]
val_labels = train_labels[:10000]

# Use the remainder of the original training data for actual training
partial_train_data = train_data[10000:]
partial_train_labels = train_labels[10000:]

In [3]:
# Scale the pixel values so they lie in the range of 0-1
partial_train_data = partial_train_data / 255.
val_data = val_data / 255.
test_data = test_data /255.

In [4]:
# Expanding the dimensions of partial_train_data, val_data and test_data to make it 4D.
partial_train_data = np.expand_dims(partial_train_data, axis=3)
val_data = np.expand_dims(val_data, axis=3)
test_data = np.expand_dims(test_data, axis=3)

In [18]:
partial_train_data.shape

(50000, 28, 28, 1)

In [5]:
# Converting the labels into one hot encoded format 
from tensorflow.keras.utils import to_categorical
partial_train_labels = to_categorical(partial_train_labels)
val_labels = to_categorical(val_labels)
test_labels = to_categorical(test_labels)

In [7]:
# Define a Sequential model for the CNN architecture.
model = Sequential([
    # Convolutional Layer 1
    Conv2D(filters=32,
           kernel_size=(3, 3),
           strides=1,
           padding='same',
           activation='relu',
           input_shape=(28, 28, 1)),

    # Convolutional Layer 2
    Conv2D(filters=32,
           kernel_size=(3, 3),
           strides=2,
           padding='valid',
           activation='relu'),

    # Convolutional Layer 3
    Conv2D(filters=64,
           kernel_size=(3, 3),
           strides=1,
           padding='same',
           activation='relu'),

    # Convolutional Layer 4
    Conv2D(filters=128,
           kernel_size=(3, 3),
           strides=1,
           padding='valid',
           activation='relu'),

    # Flatten Layer to convert the 2D feature maps to a 1D vector
    Flatten(),

    # Fully Connected Layer 1
    Dense(128, activation='relu'),

    # Output Layer with 10 units and softmax activation for classification
    Dense(10, activation='softmax')
])

# Compile the model, specifying the optimizer, loss function, and metrics.
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Define an early stopping callback to monitor the training process.
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=2)

# Train the model using the provided data and labels.
model_history = model.fit(partial_train_data,
                          partial_train_labels,
                          epochs=15,
                          batch_size=256,
                          callbacks=[callback],
                          validation_data=(val_data, val_labels),
                          verbose=1)

# Display a summary of the model's architecture.
model.summary()


Epoch 1/15
196/196 [==============================] - 49s 242ms/step - loss: 0.2659 - accuracy: 0.9198 - val_loss: 0.0710 - val_accuracy: 0.9792
Epoch 2/15
196/196 [==============================] - 47s 241ms/step - loss: 0.0494 - accuracy: 0.9849 - val_loss: 0.0487 - val_accuracy: 0.9849
Epoch 3/15
196/196 [==============================] - 48s 245ms/step - loss: 0.0289 - accuracy: 0.9907 - val_loss: 0.0423 - val_accuracy: 0.9882
Epoch 4/15
196/196 [==============================] - 49s 251ms/step - loss: 0.0191 - accuracy: 0.9941 - val_loss: 0.0760 - val_accuracy: 0.9797
Epoch 5/15
196/196 [==============================] - 49s 251ms/step - loss: 0.0119 - accuracy: 0.9963 - val_loss: 0.0520 - val_accuracy: 0.9869
Epoch 6/15
196/196 [==============================] - 49s 251ms/step - loss: 0.0096 - accuracy: 0.9967 - val_loss: 0.0380 - val_accuracy: 0.9905
Epoch 7/15
196/196 [==============================] - 47s 239ms/step - loss: 0.0067 - accuracy: 0.9978 - val_loss: 0.0473 - val_ac

In [21]:
train_images_fnn = partial_train_data.astype('float32') / 255
train_images_fnn = partial_train_data.reshape((50000, 28 * 28))

test_images_fnn = val_data.astype('float32') / 255
test_images_fnn = val_data.reshape((10000, 28 * 28))

In [25]:
sequential_7 = Sequential()
sequential_7.add(Dense(500, activation='relu', input_shape=(28 * 28,)))
sequential_7.add(Dense(100, activation='tanh', input_shape=(500,), kernel_initializer=GlorotUniform()))
sequential_7.add(Dropout(0.25))
sequential_7.add(Dense(10, activation='softmax'))

sequential_7.summary()

sequential_7.compile(optimizer=Adam(learning_rate = 0.001),
                loss='categorical_crossentropy',
                metrics=['accuracy'])

sequential_7.fit(train_images_fnn, partial_train_labels, epochs=5, batch_size=34, verbose=1)

test_loss, test_acc = sequential_7.evaluate(test_images_fnn, val_labels)
print('test_acc:', test_acc)

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_35 (Dense)            (None, 500)               392500    
                                                                 
 dense_36 (Dense)            (None, 100)               50100     
                                                                 
 dropout_10 (Dropout)        (None, 100)               0         
                                                                 
 dense_37 (Dense)            (None, 10)                1010      
                                                                 
Total params: 443610 (1.69 MB)
Trainable params: 443610 (1.69 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/5
1471/1471 [==============================] - 14s 9ms/step - loss: 0.2176 - accuracy: 0.9338
Epoch 2/5
1471/1471 [==========================

In [ ]:
313/313 [==============================] - 1s 4ms/step - loss: 0.0899 - accuracy: 0.9766
test_acc: 0.9765999913215637

In [22]:
model4 = Sequential([
    Conv2D(filters=32,
           kernel_size=(3, 3),
           strides=1,
           padding='same',
           activation='relu',
           input_shape=(28, 28, 1)),
    Conv2D(filters=32,
           kernel_size=(3, 3),
           strides=2,
           padding='valid',
           activation='relu'),
    Conv2D(filters=64,
           kernel_size=(3, 3),
           strides=1,
           padding='same',
          activation='relu'),
    Conv2D(filters=128,
           kernel_size=(3, 3),
           strides=1,
           padding='valid',
           activation='relu'),

    Flatten(),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])

model4.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=2)

model4_history = model4.fit(train_images,
                    train_labels,
                    epochs=15,
                    batch_size=256,
                    callbacks=[callback],
                    validation_data=(test_images, test_labels),
                    verbose=1)

Epoch 1/15


ValueError: in user code:

    File "C:\Users\solan\anaconda3\lib\site-packages\keras\src\engine\training.py", line 1338, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\solan\anaconda3\lib\site-packages\keras\src\engine\training.py", line 1322, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\solan\anaconda3\lib\site-packages\keras\src\engine\training.py", line 1303, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\solan\anaconda3\lib\site-packages\keras\src\engine\training.py", line 1081, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "C:\Users\solan\anaconda3\lib\site-packages\keras\src\engine\training.py", line 1139, in compute_loss
        return self.compiled_loss(
    File "C:\Users\solan\anaconda3\lib\site-packages\keras\src\engine\compile_utils.py", line 265, in __call__
        loss_value = loss_obj(y_t, y_p, samp

# Lab 3 Part 2 - Task 2: CIFAR-10 Challenge (10 Marks)

In this lab you will experiment with whatever ConvNet architecture/design you'd like on [CIFAR-10 image dataset](https://www.cs.toronto.edu/~kriz/cifar.html).


## Exercise  1: Creating the network

**Goal:** After training, your model should achieve **at least 80%** accuracy on a **validation** set within 20 epochs. (Or as close as possible as long as there is demonstrated effort to achieve this goal.)

**Data split** The training set should consist of 40000 images, the validation set should consist of 10000 images, and the test set should consist of the remaining 10000 images. **Please use the Keras `load_data()` function to import the data set.**


### Some things you can try:
- Different number/type of layers
- Different filter sizes
- Adjust the number of filters used in any given layer
- Try various pooling strategies
- Consider using batch normalization
- Check if adding regularization helps
- Consider alternative optimizers
- Try different activation functions


### Tips for training
When building/tuning your model, keep in mind the following points:

- This is experimental, so be driven by results achieved on the validation set as opposed to what you have heard/read works well or doesn't
- If the hyperparameters are working well, you should see improvement in the loss/accuracy within approximately one epoch
- For hyperparameter tuning: start by testing a large range of hyperparameters for just a few training iterations to find the combinations of parameters that are working at all
- Once you have found some sets of hyperparameters that seem to work, search more finely around these parameters. You may need to train for more epochs.
- Prefer random search to grid search for hyperparameters
- You should use the validation set for hyperparameter search and for evaluating different architectures
- The test set should only be used at the very end to evaluate your final model


In [ ]:
## Exercise 2: Describe What you did

All the work you did leading up to your final model should be summarized in this section. This should be a logical and well-organized summary of the various experiments that were tried in **Lab 3 Part 2 - Task 2:Exercise 1**, and should be captured in **table format**. Upon reading this section I should understand what you tried, the reasoning behind trying it, any quantitative values that correspond to what you tried, and the results.

See [this guide](https://www.datacamp.com/community/tutorials/markdown-in-jupyter-notebook) for how to format markdown cells in Jupyter notebooks.

## Exercise 2: Describe What you did

All the work you did leading up to your final model should be summarized in this section. This should be a logical and well-organized summary of the various experiments that were tried in **Lab 3 Part 2 - Task 2:Exercise 1**, and should be captured in **table format**. Upon reading this section I should understand what you tried, the reasoning behind trying it, any quantitative values that correspond to what you tried, and the results.

See [this guide](https://www.datacamp.com/community/tutorials/markdown-in-jupyter-notebook) for how to format markdown cells in Jupyter notebooks.